In [1]:
# import OS module for file zip/unzip

from zipfile import ZipFile
import os

In [2]:
# change to directory of zip files
working_directory = 'zipfiles'
chdir = os.chdir(working_directory)
#newdir=os.getcwd() #uncomment for troubleshooting only
#print (newdir)     # print the current directory for troubleshooting only


In [3]:
#assign the current list of files to a variable for processing
files=os.listdir()

In [4]:
# loop through each zip file and extract the file into a directory inside of zipfiles called data
for file in files:
    if file.endswith('csv.zip'):
        #print(file)
        with ZipFile(file, 'r') as f:
            f.extractall('data')
        
  

In [5]:
#create mongoDB to import data into collections for further analysis
import numpy as np
import pandas as pd
import datetime as dt
import pymongo 
import json
from pymongo import MongoClient  # remember to move this to the top

In [6]:
# create session to pymongo db to create database
#create connection strong to mongoDB database
mongoClient = pymongo.MongoClient ("mongodb://localhost:27017")


In [7]:
#read one file to determine information 
df=pd.read_csv("data/202110-citibike-tripdata.csv")

C:\Users\vertt\AppData\Local\Temp\ipykernel_29584\745428620.py:2: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("data/202110-citibike-tripdata.csv")


In [8]:
#check datatypes and convert due to error
df.dtypes

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object

In [9]:
# convert all columns in dataframe to string
convertedDF = df.astype(str)

In [10]:
#check conversion
convertedDF.dtypes

ride_id               object
rideable_type         object
started_at            object
ended_at              object
start_station_name    object
start_station_id      object
end_station_name      object
end_station_id        object
start_lat             object
start_lng             object
end_lat               object
end_lng               object
member_casual         object
dtype: object

In [11]:
convertedDF.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,DE629683A5C1F9FD,electric_bike,2021-10-09 18:16:26,2021-10-09 18:36:20,Cleveland Pl & Spring St,5492.05,E 47 St & 2 Ave,6498.10,40.722103786686034,-73.99724900722504,40.75323098,-73.97032517,member
1,B2914E13F7AD0BE7,classic_bike,2021-10-07 18:16:58,2021-10-07 18:23:42,Forsyth St\t& Grand St,5382.07,Pike St & Monroe St,5159.04,40.71779817737835,-73.99316132068634,40.71173107,-73.99193043,member
2,226BFE19A144943F,classic_bike,2021-10-29 08:57:28,2021-10-29 09:26:01,E 20 St & 2 Ave,5971.08,E 85 St & 3 Ave,7212.05,40.73587678,-73.98205027,40.77801203,-73.95407149,member
3,650246D5BC2C2C08,classic_bike,2021-10-20 08:37:39,2021-10-20 08:45:52,FDR Drive & E 35 St,6230.04,E 47 St & 2 Ave,6498.10,40.744219,-73.97121214,40.75323098,-73.97032517,member
4,BF6CDCF4CE947A03,classic_bike,2021-10-24 10:20:22,2021-10-24 10:32:12,Flatbush Ave & Ocean Ave,3704.04,Union St & Nevins St,4143.03,40.663657,-73.963014,40.67909799721684,-73.98765474557877,member


In [12]:
# read multiple files for each year into a datafile to consoldate data for each year to be able to aggregate the data

# Import libraries
import glob


# Get CSV files list from a folder
path = 'data'
csv_files = glob.glob(path + "/2018*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list = (pd.read_csv(file) for file in csv_files)

# Concatenate all DataFrames
big_df = pd.concat(df_list, ignore_index=True)

In [13]:
big_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,970,2018-01-01 13:50:57.4340,2018-01-01 14:07:08.1860,72.0,W 52 St & 11 Ave,40.767272,-73.993929,505.0,6 Ave & W 33 St,40.749013,-73.988484,31956,Subscriber,1992,1
1,723,2018-01-01 15:33:30.1820,2018-01-01 15:45:33.3410,72.0,W 52 St & 11 Ave,40.767272,-73.993929,3255.0,8 Ave & W 31 St,40.750585,-73.994685,32536,Subscriber,1969,1
2,496,2018-01-01 15:39:18.3370,2018-01-01 15:47:35.1720,72.0,W 52 St & 11 Ave,40.767272,-73.993929,525.0,W 34 St & 11 Ave,40.755942,-74.002116,16069,Subscriber,1956,1
3,306,2018-01-01 15:40:13.3720,2018-01-01 15:45:20.1910,72.0,W 52 St & 11 Ave,40.767272,-73.993929,447.0,8 Ave & W 52 St,40.763707,-73.985162,31781,Subscriber,1974,1
4,306,2018-01-01 18:14:51.5680,2018-01-01 18:19:57.6420,72.0,W 52 St & 11 Ave,40.767272,-73.993929,3356.0,Amsterdam Ave & W 66 St,40.774667,-73.984706,30319,Subscriber,1992,1


In [37]:
big_df.shape

(17548339, 15)

In [44]:
big_df.dropna(how='all', inplace=True)

In [41]:
big_df.count()

tripduration               17548339
starttime                  17548339
stoptime                   17548339
start station id           17545842
start station name         17545842
start station latitude     17548339
start station longitude    17548339
end station id             17545842
end station name           17545842
end station latitude       17548339
end station longitude      17548339
bikeid                     17548339
usertype                   17548339
birth year                 17548339
gender                     17548339
dtype: int64

In [45]:
big_df.count()

tripduration               17545842
starttime                  17545842
stoptime                   17545842
start station id           17545842
start station name         17545842
start station latitude     17545842
start station longitude    17545842
end station id             17545842
end station name           17545842
end station latitude       17545842
end station longitude      17545842
bikeid                     17545842
usertype                   17545842
birth year                 17545842
gender                     17545842
dtype: int64

In [29]:
bikeData = ["starttime", "stoptime","bikeid","tripduration","start station id","end station id"]
big_df.to_csv('../data/2018BikeData.csv', columns = bikeData, index=False)

KeyboardInterrupt: 

In [ ]:
#write all data to csv
#big_df.to_csv('../data/combined2018Data.csv', index=False)  

In [28]:
bikeStationData = ["start station id","start station latitude",
            "start station longitude","end station latitude", 
            "end station longitude","end station id"]
big_df.to_csv('../data/2018BikeStationData.csv', columns = bikeStationData, index=False)

KeyboardInterrupt: 

In [26]:
bikeUserData = ["bikeid", "gender", "usertype","birth year","gender", "starttime","stoptime"]
big_df.to_csv('../data/2018BikeUserData.csv',columns=bikeUserData,index=False)

In [ ]:
bikeData.head ()

In [ ]:
bikeStationData.head()